# <center>Curso de Modelagem de Dados para IA - PARTE 12</center>

<img src="image.jpg" alt="Drawing" style="width: 300px;"/>


## Fatiamento e filtro dos dados
Mesmo depois de limpar e pré-processar seus dados de várias maneiras, talvez você queira filtrá-los ainda mais, talvez para extrair um subconjunto dos dados para algum processamento específico. No material anterior sobre arrays e dataframes, discutimos o processo de seleção de um subconjunto de dados por indexação e fatiamento. Incluímos aqui links para documentação online:

- Manipulando índices no <a href="https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html">NumPy</a>
- Manipulando índices no <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html">Pandas</a>

### Selecionando um subconjunto de dados com base em uma condição lógica
Outro modo comum de operação em diferentes funções e pacotes do Python é a capacidade de selecionar um subconjunto de dados em um contêiner com base em alguma condição lógica especificada (ou seja, um teste ou função que retorna True ou False). Esses incluem:

- O filtro de função interno do Python, que recebe uma função e um iterável como entradas e retorna um iterador que produz os itens do iterável para os quais a função aplicada a um item é <span style="font-family: 'Courier'">True</span>.
- <a href="https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#boolean-array-indexing">Indexação de array booleano no NumPy</a>, que suporta o acesso a elementos de um array indexando com um array de *Booleans*; se alguém, por exemplo, quiser selecionar apenas os elementos de uma matriz numpy <span style="font-family: 'Courier'">x</span> que são maiores que zero, a seguinte expressão retornará uma nova matriz que satisfaça essa restrição: <span style="font-family: 'Courier'">x[x > 0]</span>. Neste exemplo, a expressão <span style="font-family: 'Courier'">x > 0</span> retorna um array de booleanos da mesma forma que x, e então <span style="font-family: 'Courier'">x[x > 0]</span> extrai o subconjunto do array para o qual <span style="font-family: 'Courier'">x > 0</span>. (Observação: se x for multidimensional, então a matriz booleana também será multidimensional, mas a extração de elementos que satisfaçam o teste resultará em uma matriz unidimensional.) A documentação online descreve <a href="https://docs.scipy.org/doc/numpy/reference/routines.indexing.html">outras rotinas de indexação NumPy</a>.
- A função <span style="font-family: 'Courier'">numpy.where</span>, que fornece um mecanismo para identificar esses índices em uma matriz numpy onde uma condição booleana específica é satisfeita ou para construir uma nova matriz com base em onde tal condição é satisfeita. Consulte a <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html#numpy.where">documentação <span style="font-family: 'Courier'">numpy.where</span></a> para obter mais informações.
- Indexação booleana em Pandas, que se baseia no mesmo mecanismo descrito acima para numpy. - Podemos selecionar subconjuntos de um dataframe que atendam a alguma condição especificada, conforme descrito em mais detalhes abaixo.

### Filtrando o Conjunto de Dados de Beisebol
Como exemplo de indexação booleana em Pandas, imagine que queremos identificar os melhores rebatedores em alguma categoria, mas não queremos que esses resultados sejam contaminados pelo pequeno tamanho da amostra, ou seja, não queremos incluir aqueles jogadores que rebateram tão pouco que suas estatísticas não são confiáveis. Em Pandas, podemos facilmente impor um corte, baseado em um número mínimo de *at-bats* (AB), indexando na condição <span style="font-family: 'Courier'">pl_bat.AB >= min_AB</span>. Por exemplo, se quiséssemos listar os jogadores com maior porcentagem de slugging de todos os tempos ('SLG'), poderíamos emitir o seguinte comando:

In [1]:
import glob, os
import pandas as pd

In [2]:
def read_all_databank_core_csv(directory):
    """
    read all csv files in the specified baseball databank directory and
    populate a dictionary storing each of the tables keyed to its name
    """
    dfs = {}
    files = glob.glob('{}/*.csv'.format(directory))
    for f in files:
        d, name = os.path.split(f)
        table = os.path.splitext(name)[0]
        df = pd.read_csv(f)
        dfs[table] = df
    return dfs

bbdfs = read_all_databank_core_csv('data/baseballdatabank/core')

In [3]:
batting = bbdfs['Batting']
pitching = bbdfs['Pitching']
teams = bbdfs['Teams']

In [4]:
batting['1B'] = batting['H'] - batting['2B'] - batting['3B'] - batting['HR']
teams['1B'] = teams['H'] - teams['2B'] - teams['3B'] - teams['HR']

In [5]:
batting_by_year = batting.groupby('yearID').sum().reset_index()

In [6]:
pl_bat = batting.groupby('playerID').sum().reset_index()

In [7]:
pl_bat['BA'] = pl_bat['H'] / pl_bat['AB']
pl_bat['OBP'] = (pl_bat['H']+pl_bat['BB']+pl_bat['HBP']) / (pl_bat['AB']+pl_bat['BB']+pl_bat['HBP']+pl_bat['SF'])
pl_bat['SLG'] = (pl_bat['1B']+2*pl_bat['2B']+3*pl_bat['3B']+4*pl_bat['HR']) / pl_bat['AB']
pl_bat['OPS'] = (pl_bat['OBP']+pl_bat['SLG'])

In [8]:
pl_bat[pl_bat.AB >= 100].sort_values(by='SLG', ascending=False).head(30)

,playerID,yearID,stint,G,AB,R,H,2B,3B,HR,...,IBB,HBP,SH,SF,GIDP,1B,BA,OBP,SLG,OPS
15454,ruthba01,42339,22,2503,8398,2174,2873,506,136,714,...,0.0,43.0,113.0,0.0,2.0,1517,0.342105,0.473960,0.689807,1.163767
307,alvaryo01,2019,1,87,313,58,98,26,0,27,...,4.0,2.0,0.0,2.0,9.0,45,0.313099,0.411924,0.654952,1.066876
19120,willite01,37057,19,2292,7706,1798,2654,525,71,521,...,86.0,39.0,5.0,20.0,197.0,1537,0.344407,0.481709,0.633792,1.115500
6273,gehrilo01,32827,17,2164,8001,1888,2721,534,163,493,...,0.0,45.0,106.0,0.0,2.0,1531,0.340082,0.447352,0.632421,1.079773
5816,foxxji01,40634,22,2317,8134,1751,2646,458,125,534,...,0.0,13.0,71.0,0.0,69.0,1529,0.325301,0.428274,0.609294,1.037568
1621,bondsba01,43923,22,2986,9847,2227,2935,601,77,762,...,688.0,106.0,4.0,91.0,165.0,1495,0.298060,0.444295,0.606885,1.051180
6828,greenha01,25201,13,1394,5193,1051,1628,379,71,331,...,0.0,16.0,35.0,0.0,66.0,847,0.313499,0.411813,0.605045,1.016858
941,bassjo01,5620,3,25,100,19,29,2,10,3,...,0.0,0.0,0.0,0.0,0.0,14,0.290000,0.310680,0.600000,0.910680
17472,tatisfe02,2019,1,84,334,61,106,13,6,22,...,1.0,5.0,0.0,3.0,4.0,65,0.317365,0.379032,0.589820,0.968853
11586,mcgwima01,33893,18,1874,6187,1167,1626,252,6,583,...,150.0,75.0,3.0,78.0,147.0,785,0.262809,0.394149,0.588169,0.982318


A expressão <span style="font-family: 'Courier'">pl_bat.AB >= 100</span> cria uma série Pandas de booleanos (dependendo se <span style="font-family: 'Courier'">AB >=100</span> para uma determinada linha) e, em seguida, usa isso para selecionar apenas o subconjunto de linhas em pl_bat para o qual essa série é verdadeira (<span style="font-family: 'Courier'">True</span>) e retorna um novo dataframe. Nesse dataframe reduzido, podemos chamar métodos adicionais, como <span style="font-family: 'Courier'">sort_values</span>.

### Filtrando o Dataset sobre Incêndios Florestais
Anteriormente, fizemos um pré-processamento no Dataset sobre Incêndios Florestais para limpar alguns dos dados originais e obter <span style="font-family: 'Courier'">ALARM_DATEs</span> e <span style="font-family: 'Courier'">CONT_DATEs</span> representados consistentemente como objetos de data e hora. Junto com nossas habilidades em aumento de dados, podemos criar uma nova coluna de dados representando a duração temporal geral de um incêndio. Como as colunas <span style="font-family: 'Courier'">CONT_DATE</span> e <span style="font-family: 'Courier'">ALARM_DATE</span> contêm objetos <span style="font-family: 'Courier'">datetime64</span>, podemos subtrair um do outro, resultando em um objeto <span style="font-family: 'Courier'">timedelta</span>. A partir disso, podemos extrair o número de dias após o alarme soar até quando o fogo foi contido e inspecionar o intervalo dessas durações (ou seja, a duração mínima e máxima):

In [10]:
import pandas as pd

In [16]:
df16 = pd.read_excel('data/wildfires/Fires_100.xlsx', sheet_name='2016',na_values=['<Null>'], converters= {'ALARM_DATE': pd.to_datetime, 'CONT_DATE': pd.to_datetime})
df17 = pd.read_excel('data/wildfires/Fires_100.xlsx', sheet_name='2017', na_values=['<Null>'], 
                   converters= {'ALARM_DATE': pd.to_datetime, 'CONT_DATE': pd.to_datetime})

In [17]:
df17.rename(columns={'YEAR_': 'YEAR'}, inplace=True)

In [18]:
dffires = pd.concat((df16, df17), axis=0)

In [19]:
dffires.head()

,YEAR,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE
0,1878.0,California,Contract County,Los Angeles County,,0,NaT,NaT,14 - Unknown / Unidentified,NaN,NaN,59468.8800,NaN,Suppression (Wildfire)
1,1895.0,California,Contract County,Los Angeles County,,0,NaT,NaT,14 - Unknown / Unidentified,NaN,NaN,628.1136,NaN,Suppression (Wildfire)
2,1896.0,California,Contract County,Los Angeles County,,0,NaT,NaT,14 - Unknown / Unidentified,NaN,NaN,7531.7170,NaN,Suppression (Wildfire)
3,1898.0,California,Contract County,Los Angeles County,,0,NaT,NaT,14 - Unknown / Unidentified,NaN,NaN,1660.1030,NaN,Suppression (Wildfire)
4,1898.0,California,Contract County,Los Angeles County,,0,NaT,NaT,14 - Unknown / Unidentified,NaN,NaN,428.4387,NaN,Suppression (Wildfire)


In [20]:
dffires['DURATION'] = (dffires['CONT_DATE'] - dffires['ALARM_DATE']).dt.days
print(dffires['DURATION'].min(), dffires['DURATION'].max())

-32862.0 3319.0


Curiosamente, podemos ver que alguns dos incêndios estão listados como tendo durações negativas, ou seja, foram contidos antes mesmo de serem identificados, o que provavelmente é resultado de algum erro de entrada de dados. Se fôssemos executar <span style="font-family: 'Courier'">dffires[dffires.DURATION < 0]</span>. pudemos ver o subconjunto de incêndios para o qual este é o caso (há 9 ao todo). Também podemos ver nos resultados acima que há um incêndio que supostamente durou mais de 3.000 dias (9 anos), provavelmente também devido a um erro de entrada de dados. Dependendo de como você deseja trabalhar com esse conjunto de dados, talvez queira tentar corrigir esses erros, mas geralmente é mais fácil jogá-los fora. Podemos fazer isso filtrando o dataframe para que possamos prosseguir com uma análise mais aprofundada. Esta operação de filtragem inclui apenas as linhas com <span style="font-family: 'Courier'">DURATION >=0</span> e <span style="font-family: 'Courier'">DURATION < 3000</span>:

In [21]:
dffires = dffires[(dffires.DURATION >= 0) & (dffires.DURATION < 3000)]

In [22]:
dffires.head()

,YEAR,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE,DURATION
1214,1921.0,California,USDA Forest Service,Eldorado National Forest,PENNSYLVANIA,0,1921-10-04,1921-10-04,9 - Miscellaneous,Containment 'fire_day' estimated,NaN,273.4315,NaN,Suppression (Wildfire),0.0
1251,1921.0,California,National Park Service,Sequoia - Kings Canyon NP,ELK CREEK,0,1921-07-05,1921-07-17,4 - Campfire,NaN,1600.0,1551.4670,8 - Unknown,Suppression (Wildfire),12.0
1340,1922.0,California,National Park Service,Sequoia - Kings Canyon NP,PANTHER,0,1922-09-19,1922-09-23,1 - Lightning,NaN,350.0,535.8748,8 - Unknown,Suppression (Wildfire),4.0
1341,1922.0,California,National Park Service,Sequoia - Kings Canyon NP,HOSPITAL,0,1922-08-13,1922-08-17,4 - Campfire,NaN,550.0,667.1011,8 - Unknown,Suppression (Wildfire),4.0
1342,1922.0,California,National Park Service,Sequoia - Kings Canyon NP,E.FORK,0,1922-07-04,1922-07-06,4 - Campfire,NaN,300.0,449.5453,8 - Unknown,Suppression (Wildfire),2.0
